<a href="https://colab.research.google.com/github/annefrancine/DataScience_Estiam/blob/main/for%C3%AAts_al%C3%A9atoires.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Méthodes ensemble** : Ensemble learning : 
En machine learning, la méthode ensembliste consiste à utiliser plusieurs algorithmes d'apprentissage automatique, en les mettant en commun pour obtenir des prédictions de meilleure qualité 


 L'ensemble de méthodes bagging, boosting et les forêts aléatoires est un exemple d'apprentissage ensembliste

Dans un premier temps, étudions le dataset à notre disposition : le "Human Activity Recognition Using Smartphones Data Set".

Ce jeu de données contient les logs de capteurs de smartphone sur une trentaine d'individus en train d'effectuer des activités (s'assoir, se mettre debout, marcher, etc). L'objectif sera de prédire à partir des logs de capteurs le type d'activités que le sujet est en train d'effectuer.

En regardant le fichier de description du dataset, on peut observer qu'il y a beaucoup de features (561). D'emblée, en observant ce qu'elles désignent, on peut se dire qu'il y a une certaine redondance entre toutes ces variables. Dans un premier temps, on va effectuer une modélisation "brute" sans se soucier de nettoyer le jeu de données.

Dans un second temps, on va utiliser cette première modélisation pour mieux comprendre le dataset et ainsi effectuer une seconde modélisation plus efficace en éliminant des variables peu importantes.

On commence par charger les données.

In [1]:
from google.colab import files
files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv


In [2]:
import pandas as pd

train = pd.read_csv("train.csv")
test  = pd.read_csv("test.csv")

X_train = train[train.columns[:-2]]
y_train = train['Activity']

X_test = test[test.columns[:-2]]
y_test = test['Activity']

In [3]:
print(train.shape)

(7352, 563)


Une fois les données chargées, on peut déclarer un nouveau modèle de forêts aléatoires pour la classification logiquement appelé dans scikit   sklearn.RandomForestClassifier . On définit comme hyperparamètres 500 pour le nombre d'arbres. 

In [5]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=500, oob_score=True)

In [6]:
model = rfc.fit(X_train, y_train)

In [7]:
#performance
from sklearn.metrics import accuracy_score

pred = rfc.predict(X_test)
print("accuracy {:.2f}".format(accuracy_score(y_test, pred)))

accuracy 0.93


Donc en utilisant  SelectFromModel  avec un seuil d'importance choisi à l'aide de l'argument  threshold  , on peut créer une sélection des features qui sont les plus importantes à la création d'un modèle.

In [8]:
from sklearn.feature_selection import SelectFromModel
select = SelectFromModel(rfc, prefit=True, threshold=0.003)
X_train2 = select.transform(X_train)
print(X_train2.shape)

(7352, 82)


On a divisé par 5 le nombre de features utilisées, pas mal mais voyons si les performances restent similaires. À l'aide de l'argument threshold, on peut choisir le seuil d'importance que l'on souhaite pour les features à sélectionner.  On calcule en même temps le gain de temps car c'est ce qui nous intéresse principalement dans l'amélioration des performances

In [9]:
import timeit

rfc2 = RandomForestClassifier(n_estimators=500, oob_score=True)

start_time = timeit.default_timer()

rfc2 = rfc2.fit(X_train2, y_train)

X_test2 = select.transform(X_test)

pred = rfc2.predict(X_test2)
elapsed = timeit.default_timer() - start_time
accuracy = accuracy_score(y_test, pred)

print("accuracy {:.2f} time {:.2f}s".format(accuracy, elapsed))


accuracy 0.89 time 22.16s
